In [6]:
import uuid
from langserve import RemoteRunnable

chat = RemoteRunnable("http://localhost:8000/")

inputs = {
   "input": {
        "input":"bạn là ai?",
        "chat_history":[
            {"role":"system",
             "content": "Bạn là đại học Nguyễn Tất Thành"},
             {"role":"human",
              "content":"bạn có phải là người không?"}
        ]
}}

sesssion_id = str(uuid.uuid4())

chat.invoke(
    inputs,
    {"configurable":{"session_id": sesssion_id}}
)

HTTPStatusError: Client error '422 Unprocessable Entity' for url 'http://localhost:8000/invoke'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/422 for {"detail":[{"loc":["chat_history"],"msg":"field required","type":"value_error.missing"}]}

In [2]:
from langchain_core.runnables import RunnableLambda

def f(x: str) -> str:
    return x + "a"

def g(x: str) -> str:
    return x + "z"

runnable = RunnableLambda(f) | g
as_tool = runnable.as_tool()
as_tool.invoke("b")

'baz'

In [ ]:
from typing import Any, Dict, List, 


def format_docs(docs: List[Document]) -> str:
    '''Format the docs.'''
    return ", ".join([doc.page_content for doc in docs])

format_docs = RunnableLambda(format_docs)

In [3]:
from langchain_community.tools import tool

@tool
def some_tool(x: int, y: str) -> dict:
    '''Some_tool.'''
    return {"x": x, "y": y}

In [6]:

from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [("system", "You are Cat Agent 007"), ("human", "{question}")]
).with_config({"run_name": "my_template", "tags": ["my_template"]})

In [8]:
from langchain_core.runnables import RunnableLambda

async def reverse(s: str) -> str:
    return s[::-1]

chain = RunnableLambda(func=reverse)

events = [
    event async for event in chain.astream_events("hello", version="v2")
]

# will produce the following events (run_id, and parent_ids
# has been omitted for brevity):
[
    {
        "data": {"input": "hello"},
        "event": "on_chain_start",
        "metadata": {},
        "name": "reverse",
        "tags": [],
    },
    {
        "data": {"chunk": "olleh"},
        "event": "on_chain_stream",
        "metadata": {},
        "name": "reverse",
        "tags": [],
    },
    {
        "data": {"output": "olleh"},
        "event": "on_chain_end",
        "metadata": {},
        "name": "reverse",
        "tags": [],
    },
]

[{'data': {'input': 'hello'},
  'event': 'on_chain_start',
  'metadata': {},
  'name': 'reverse',
  'tags': []},
 {'data': {'chunk': 'olleh'},
  'event': 'on_chain_stream',
  'metadata': {},
  'name': 'reverse',
  'tags': []},
 {'data': {'output': 'olleh'},
  'event': 'on_chain_end',
  'metadata': {},
  'name': 'reverse',
  'tags': []}]

In [9]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda, RunnableConfig
import asyncio


async def slow_thing(some_input: str, config: RunnableConfig) -> str:
    """Do something that takes a long time."""
    await asyncio.sleep(1) # Placeholder for some slow operation
    await adispatch_custom_event(
        "progress_event",
        {"message": "Finished step 1 of 3"},
        config=config # Must be included for python < 3.10
    )
    await asyncio.sleep(1) # Placeholder for some slow operation
    await adispatch_custom_event(
        "progress_event",
        {"message": "Finished step 2 of 3"},
        config=config # Must be included for python < 3.10
    )
    await asyncio.sleep(1) # Placeholder for some slow operation
    return "Done"

slow_thing = RunnableLambda(slow_thing)

async for event in slow_thing.astream_events("some_input", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'some_input'}, 'name': 'slow_thing', 'tags': [], 'run_id': '85989423-49bb-4ba7-8e82-54191d65e81a', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': '85989423-49bb-4ba7-8e82-54191d65e81a', 'name': 'progress_event', 'tags': [], 'metadata': {}, 'data': {'message': 'Finished step 1 of 3'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': '85989423-49bb-4ba7-8e82-54191d65e81a', 'name': 'progress_event', 'tags': [], 'metadata': {}, 'data': {'message': 'Finished step 2 of 3'}, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': '85989423-49bb-4ba7-8e82-54191d65e81a', 'name': 'slow_thing', 'tags': [], 'metadata': {}, 'data': {'chunk': 'Done'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'Done'}, 'run_id': '85989423-49bb-4ba7-8e82-54191d65e81a', 'name': 'slow_thing', 'tags': [], 'metadata': {}, 'parent_ids': []}
